In [91]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, lit, split
from configparser import ConfigParser
import psycopg2
from IPython.display import display
import pprint as pp

In [34]:
# create a spark session
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Wake County Restaurants") \
    .getOrCreate()

# create a dataframe from the CSV
df = spark \
    .read \
    .format("csv") \
    .option("header", "true") \
    .load("../data/Restaurants_in_Wake_County_NC.csv")

df.show(5)

df.printSchema()
df.count()

+--------+-----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+--------------------+-----------------+--------+------------+-----------+-------------+
|OBJECTID|     HSISID|                NAME|            ADDRESS1|ADDRESS2|       CITY|STATE|POSTALCODE|   PHONENUMBER|  RESTAURANTOPENDATE|     FACILITYTYPE|PERMITID|           X|          Y|GEOCODESTATUS|
+--------+-----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+--------------------+-----------------+--------+------------+-----------+-------------+
|    1001|04092016024|                WABA|2502 1/2 HILLSBOR...|    null|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18T00:00:...|       Restaurant|    6952|-78.66818477|35.78783803|            M|
|    1002|04092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    null|       CARY|   NC|     27518|(919) 852-6651|2011-11-08T00:00:...|       Food Stand|    6953|-78.78211173|35

3440

In [35]:
# # create a new column
# df.withColumn()

# # rename an existing column
# df.withColumnRenamed()

# # get a column by name
# df.col()
# df.drop()
# df.lit()
# df.concat()

# Displays the top 5 rows. Accepts an optional int parameter - num. of rows to show
# df.head()

# Similar to head, but displays the last rows
# df.tail()

# The dimensions of the dataframe as a (rows, cols) tuple
# df.shape

# The number of columns. Equal to df.shape[0]
# len(df) 

# An array of the column names
# df.columns 

# Columns and their types
# df.dtypes

# Converts the frame to a two-dimensional table
# df.values 

# Displays descriptive stats for all columns
# df.describe()


In [36]:
# add, remap and drop columns
df = df.withColumn("county", lit("Wake")) \
    .withColumnRenamed("HSISID", "datasetId") \
    .withColumnRenamed("NAME", "name") \
    .withColumnRenamed("ADDRESS1", "address1") \
    .withColumnRenamed("ADDRESS2", "address2") \
    .withColumnRenamed("CITY", "city") \
    .withColumnRenamed("STATE", "state") \
    .withColumnRenamed("POSTALCODE", "zip") \
    .withColumnRenamed("PHONENUMBER", "tel") \
    .withColumnRenamed("RESTAURANTOPENDATE", "dateStart") \
    .withColumnRenamed("FACILITYTYPE", "type") \
    .withColumnRenamed("X", "geoX") \
    .withColumnRenamed("Y", "geoY") \
    .drop("OBJECTID") \
    .drop("PERMITID") \
    .drop("GEOCODESTATUS")

df.head()

Row(datasetId='04092016024', name='WABA', address1='2502 1/2 HILLSBOROUGH  ST ', address2=None, city='RALEIGH', state='NC', zip='27607', tel='(919) 833-1710', dateStart='2011-10-18T00:00:00.000Z', type='Restaurant', geoX='-78.66818477', geoY='35.78783803', county='Wake')

In [37]:
# create a unique id for each record
delim = lit("_")
df = df.withColumn("id", concat(df["state"], delim, df["county"], delim, df["datasetId"]))

df.head()
df.printSchema()

root
 |-- datasetId: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address1: string (nullable = true)
 |-- address2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- tel: string (nullable = true)
 |-- dateStart: string (nullable = true)
 |-- type: string (nullable = true)
 |-- geoX: string (nullable = true)
 |-- geoY: string (nullable = true)
 |-- county: string (nullable = false)
 |-- id: string (nullable = true)



In [43]:
# Returns the number of partitions in RDD
df.rdd.getNumPartitions()

# increase the level of parallelism in this RDD
df = df.repartition(4)

df.rdd.getNumPartitions()

4

In [66]:
df.schema.simpleString()
pp.pprint(df.schema.jsonValue())

{'fields': [{'metadata': {},
             'name': 'datasetId',
             'nullable': True,
             'type': 'string'},
            {'metadata': {},
             'name': 'name',
             'nullable': True,
             'type': 'string'},
            {'metadata': {},
             'name': 'address1',
             'nullable': True,
             'type': 'string'},
            {'metadata': {},
             'name': 'address2',
             'nullable': True,
             'type': 'string'},
            {'metadata': {},
             'name': 'city',
             'nullable': True,
             'type': 'string'},
            {'metadata': {},
             'name': 'state',
             'nullable': True,
             'type': 'string'},
            {'metadata': {}, 'name': 'zip', 'nullable': True, 'type': 'string'},
            {'metadata': {}, 'name': 'tel', 'nullable': True, 'type': 'string'},
            {'metadata': {},
             'name': 'dateStart',
             'nullable': True,
    

In [87]:
dfJSON = spark \
    .read \
    .format("json") \
    .load("../data/Restaurants_in_Durham_County_NC.json")


display(dfJSON.show(5))
pp.pprint(dfJSON.head(1))

+----------------+--------------------+--------------------+--------------------+--------------------+
|       datasetid|              fields|            geometry|    record_timestamp|            recordid|
+----------------+--------------------+--------------------+--------------------+--------------------+
|restaurants-data|[, Full-Service R...|[[-78.9573299, 35...|2017-07-13T09:15:...|1644654b953d1802c...|
|restaurants-data|[, Nursing Home, ...|[[-78.8895483, 36...|2017-07-13T09:15:...|93573dbf8c9e799d8...|
|restaurants-data|[, Fast Food Rest...|[[-78.9593263, 35...|2017-07-13T09:15:...|0d274200c7cef50d0...|
|restaurants-data|[, Full-Service R...|[[-78.9060312, 36...|2017-07-13T09:15:...|cf3e0b175a6ebad2a...|
|restaurants-data|[,, [36.0556347, ...|[[-78.9135175, 36...|2017-07-13T09:15:...|e796570677f7c39cc...|
+----------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



None

[Row(datasetid='restaurants-data', fields=Row(closing_date=None, est_group_desc='Full-Service Restaurant', geolocation=[35.9207272, -78.9573299], hours_of_operation=None, id='56060', insp_freq=4, opening_date='1994-09-01', premise_address1='4711 HOPE VALLEY RD', premise_address2='SUITE 6C', premise_city='DURHAM', premise_name='WEST 94TH ST PUB', premise_phone='(919) 403-0025', premise_state='NC', premise_zip='27707', risk=4, rpt_area_desc='Food Service', seats=60, sewage='3 - Municipal/Community', smoking_allowed='NO', status='ACTIVE', transitional_type_desc='FOOD', type_description='1 - Restaurant', water='5 - Municipal/Community'), geometry=Row(coordinates=[-78.9573299, 35.9207272], type='Point'), record_timestamp='2017-07-13T09:15:31-04:00', recordid='1644654b953d1802c3c941211f61be1f727b2951')]


In [72]:
dfJSON.printSchema()

root
 |-- datasetid: string (nullable = true)
 |-- fields: struct (nullable = true)
 |    |-- closing_date: string (nullable = true)
 |    |-- est_group_desc: string (nullable = true)
 |    |-- geolocation: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- hours_of_operation: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- insp_freq: long (nullable = true)
 |    |-- opening_date: string (nullable = true)
 |    |-- premise_address1: string (nullable = true)
 |    |-- premise_address2: string (nullable = true)
 |    |-- premise_city: string (nullable = true)
 |    |-- premise_name: string (nullable = true)
 |    |-- premise_phone: string (nullable = true)
 |    |-- premise_state: string (nullable = true)
 |    |-- premise_zip: string (nullable = true)
 |    |-- risk: long (nullable = true)
 |    |-- rpt_area_desc: string (nullable = true)
 |    |-- seats: long (nullable = true)
 |    |-- sewage: string (nullable = true)
 |   

In [101]:
dfJSON = dfJSON.withColumn("county", lit("Durham")) \
    .withColumn("datasetId", dfJSON["fields.id"]) \
    .withColumn("name", dfJSON["fields.premise_name"]) \
    .withColumn("address1", dfJSON["fields.premise_address1"]) \
    .withColumn("address2", dfJSON["fields.premise_address2"]) \
    .withColumn("city", dfJSON["fields.premise_city"]) \
    .withColumn("state", dfJSON["fields.premise_state"]) \
    .withColumn("zip", dfJSON["fields.premise_zip"]) \
    .withColumn("tel", dfJSON["fields.premise_phone"]) \
    .withColumn("dateStart", dfJSON["fields.opening_date"]) \
    .withColumn("dateEnd", dfJSON["fields.closing_date"]) \
    .withColumn("type", split(dfJSON["fields.type_description"], " - ").getItem(1)) \
    .withColumn("geoX", dfJSON["fields.geolocation"].getItem(0)) \
    .withColumn("geoY", dfJSON["fields.geolocation"].getItem(1)) \
    .drop("geometry") \
    .drop("fields")

dfJSON.head()

Row(datasetId='56060', record_timestamp='2017-07-13T09:15:31-04:00', recordid='1644654b953d1802c3c941211f61be1f727b2951', county='Durham', name='WEST 94TH ST PUB', address1='4711 HOPE VALLEY RD', address2='SUITE 6C', city='DURHAM', state='NC', zip='27707', tel='(919) 403-0025', dateStart='1994-09-01', dateEnd=None, type='Restaurant', geoX=35.9207272, geoY=-78.9573299, id='NC_Durham_56060')

In [104]:
delim = lit("_")
dfJSON = dfJSON.withColumn("id", concat(dfJSON["state"], delim, dfJSON["county"], delim, dfJSON["datasetId"]))

dfJSON.show(5)

+---------+--------------------+--------------------+------+--------------------+--------------------+--------+------+-----+-----+--------------+----------+-------+--------------------+----------+-----------+---------------+
|datasetId|    record_timestamp|            recordid|county|                name|            address1|address2|  city|state|  zip|           tel| dateStart|dateEnd|                type|      geoX|       geoY|             id|
+---------+--------------------+--------------------+------+--------------------+--------------------+--------+------+-----+-----+--------------+----------+-------+--------------------+----------+-----------+---------------+
|    56060|2017-07-13T09:15:...|1644654b953d1802c...|Durham|    WEST 94TH ST PUB| 4711 HOPE VALLEY RD|SUITE 6C|DURHAM|   NC|27707|(919) 403-0025|1994-09-01|   null|          Restaurant|35.9207272|-78.9573299|NC_Durham_56060|
|    58123|2017-07-13T09:15:...|93573dbf8c9e799d8...|Durham|BROOKDALE DURHAM IFS|4434 BEN FRANKLIN..

In [103]:
dfJSON.printSchema()

root
 |-- datasetId: string (nullable = true)
 |-- record_timestamp: string (nullable = true)
 |-- recordid: string (nullable = true)
 |-- county: string (nullable = false)
 |-- name: string (nullable = true)
 |-- address1: string (nullable = true)
 |-- address2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- tel: string (nullable = true)
 |-- dateStart: string (nullable = true)
 |-- dateEnd: string (nullable = true)
 |-- type: string (nullable = true)
 |-- geoX: double (nullable = true)
 |-- geoY: double (nullable = true)
 |-- id: string (nullable = true)



In [105]:
dfJSON = dfJSON.repartition(4)
dfJSON.rdd.getNumPartitions()

4